### Pre process and merge the data

#### Import packages

In [29]:
import pandas as pd
import numpy as np

#### Load the datasets

In [2]:
campaign_df = pd.read_csv("../data/campaign_data.csv")
creative_df = pd.read_csv("../data/creative_data.csv")
inventory_df = pd.read_csv("../data/Inventory_data.csv")

##### Fix inconsistency namings

In [4]:
campaign_df.rename(columns={'campaignid': 'campaign_id'}, inplace=True)

#### Remove rows that have a campaign but no inventory

In [5]:
# Get campaign_ids that were removed
removed_campaign_ids = campaign_df[~campaign_df['campaign_id'].isin(inventory_df['campaign_id'])]['campaign_id']

# Filter out rows where campaign_id exists in campaign_df but not in inventory_df
removed_rows = campaign_df[campaign_df['campaign_id'].isin(removed_campaign_ids)]

In [6]:
removed_rows['campaign_id']

9     5pk0f4n
12    2la80vo
15    82k2wx4
25    xplk7p6
40    1p1ksf7
44    ifdayp2
74    ejj5vbv
79    377r1eu
Name: campaign_id, dtype: object

In [7]:
# Get campaign_ids that were removed
removed_campaign_ids = campaign_df[~campaign_df['campaign_id'].isin(inventory_df['campaign_id'])]['campaign_id']

# Filter out rows where campaign_id exists in campaign_df but not in creative df
removed_rows = campaign_df[campaign_df['campaign_id'].isin(removed_campaign_ids)]

In [8]:
removed_rows['campaign_id']

9     5pk0f4n
12    2la80vo
15    82k2wx4
25    xplk7p6
40    1p1ksf7
44    ifdayp2
74    ejj5vbv
79    377r1eu
Name: campaign_id, dtype: object

In [9]:
removed_rows.to_csv("../data/campaign_with_no_inventory.csv")

##### Merge the 3 dataframes based on campaign_ig and game_key

In [10]:
import pandas as pd

# Merge the dataframes based on common columns
merged_df = pd.merge(campaign_df, creative_df, on='campaign_id', how='left')
merged_df = pd.merge(merged_df, inventory_df, on=['campaign_id', 'game_key'], how='left')

# Print the merged dataframe
print(merged_df.head())

  campaign_id  budget_amount budget_currencycode         KPI pricing_model  \
0     f4p5a01         277.12                 USD  Engagement           CPE   
1     f4p5a01         277.12                 USD  Engagement           CPE   
2     f4p5a01         277.12                 USD  Engagement           CPE   
3     f4p5a01         277.12                 USD  Engagement           CPE   
4     f4p5a01         277.12                 USD  Engagement           CPE   

  geo_targeting vertical                                          targeting  \
0           NaN   Sports  Opinion Leaders| Investors| Policy Makers| C&I...   
1           NaN   Sports  Opinion Leaders| Investors| Policy Makers| C&I...   
2           NaN   Sports  Opinion Leaders| Investors| Policy Makers| C&I...   
3           NaN   Sports  Opinion Leaders| Investors| Policy Makers| C&I...   
4           NaN   Sports  Opinion Leaders| Investors| Policy Makers| C&I...   

                                            game_key adf

In [11]:
merged_df.columns

Index(['campaign_id', 'budget_amount', 'budget_currencycode', 'KPI',
       'pricing_model', 'geo_targeting', 'vertical', 'targeting', 'game_key',
       'adformat', 'creative', 'renderingcontext', 'osfamily', 'devicetype',
       'devicemake', 'site_name', 'matchedfoldposition', 'browser',
       'impression', 'engagement', 'click'],
      dtype='object')

In [12]:
len(merged_df)

2975312

In [13]:
# Filter out rows where campaign_id exists in campaign_df but not in inventory_df
merged_df = merged_df[~merged_df['campaign_id'].isin(campaign_df[~campaign_df['campaign_id'].isin(inventory_df['campaign_id'])]['campaign_id'])]

In [14]:
len(merged_df)

2975304

In [15]:
merged_df.columns

Index(['campaign_id', 'budget_amount', 'budget_currencycode', 'KPI',
       'pricing_model', 'geo_targeting', 'vertical', 'targeting', 'game_key',
       'adformat', 'creative', 'renderingcontext', 'osfamily', 'devicetype',
       'devicemake', 'site_name', 'matchedfoldposition', 'browser',
       'impression', 'engagement', 'click'],
      dtype='object')

#### Compute click through rate and engagement rate

In [24]:

# Calculate CTR (Click-Through Rate)
merged_df['CTR'] = np.where(merged_df['impression'] != 0,
                            (merged_df['click'] / merged_df['impression']) * 100,
                            0)

# Calculate ER (Engagement Rate)
merged_df['ER'] = np.where(merged_df['impression'] != 0,
                           (merged_df['engagement'] / merged_df['impression']) * 100,
                           0)

#### Mapping of number to value for 3 colums

In [30]:
# Mapping dictionary for osfamily
os_family_mapping = {
    1: 'Other',
    2: 'Windows',
    3: 'OS X',
    4: 'Linux',
    5: 'iOS',
    6: 'Android',
    7: 'Windows Phone'
}

# Map numerical values to categories
merged_df['osfamily'] = merged_df['osfamily'].map(os_family_mapping)

In [31]:
# Mapping dictionary for devicetype
device_type_mapping = {
    1: 'Other',
    2: 'PC',
    3: 'Tablet',
    4: 'Mobile',
    5: 'Roku',
    6: 'ConnectedTV',
    7: 'OutOfHome',
    8: 'HomeAssistant'
}

# Map numerical values to categories
merged_df['devicetype'] = merged_df['devicetype'].map(device_type_mapping)

In [32]:
# Mapping dictionary for fold position
foldposition_mapping = {
    1: 'Any',
    2: 'Above',
    3: 'Below',
    4: 'Unknown'
}

# Map numerical values to categories
merged_df['matchedfoldposition'] = merged_df['matchedfoldposition'].map(foldposition_mapping)

#### Handle missing values

In [33]:
# Calculate the percentage of missing values in each column
missing_percentage = merged_df.isnull().sum() / len(merged_df) * 100

# Print the percentage of missing values for each column
print(missing_percentage)

campaign_id             0.000000
budget_amount           0.000000
budget_currencycode     0.000000
KPI                     0.000000
pricing_model           0.000000
geo_targeting          91.161239
vertical                0.000000
targeting              81.240472
game_key                0.000000
adformat                0.000000
creative                2.900846
renderingcontext        0.000000
osfamily                0.003899
devicetype              0.000101
devicemake              1.916644
site_name               0.000000
matchedfoldposition     0.000067
browser                 0.091890
impression              0.000000
engagement              0.000000
click                   0.000000
CTR                     0.000000
ER                      0.000000
dtype: float64


In [34]:
merged_df['geo_targeting'].fillna(value="All Country", inplace=True)
merged_df['targeting'].fillna(value="All", inplace=True)
merged_df['creative'].fillna(value="Unknown", inplace=True)
merged_df['browser'].fillna(value=merged_df['browser'].mode()[0], inplace=True)
merged_df['devicemake'].fillna(value=merged_df['devicemake'].mode()[0], inplace=True)
merged_df['devicetype'].fillna(value=merged_df['devicetype'].mode()[0], inplace=True)
merged_df['matchedfoldposition'].fillna(value=merged_df['matchedfoldposition'].mode()[0], inplace=True)
merged_df['osfamily'].fillna(value=merged_df['osfamily'].mode()[0], inplace=True)

In [35]:
# Calculate the percentage of missing values in each column
missing_percentage = merged_df.isna().sum() / len(merged_df) * 100

# Print the percentage of missing values for each column
print(missing_percentage)

campaign_id            0.0
budget_amount          0.0
budget_currencycode    0.0
KPI                    0.0
pricing_model          0.0
geo_targeting          0.0
vertical               0.0
targeting              0.0
game_key               0.0
adformat               0.0
creative               0.0
renderingcontext       0.0
osfamily               0.0
devicetype             0.0
devicemake             0.0
site_name              0.0
matchedfoldposition    0.0
browser                0.0
impression             0.0
engagement             0.0
click                  0.0
CTR                    0.0
ER                     0.0
dtype: float64


#### Save the merged dta

In [29]:
merged_df.to_csv("../data/merged_data.csv")